In [28]:
import pandas as pd
import requests

In [29]:
dfpostpath = "postAITS4.csv"
followerpath  = "followers4.csv"

In [30]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [31]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [32]:
df = pd.read_csv(followerpath)
df

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username,follower_count
0,6.315919e+10,theartcandee,6.341463e+10,anthonyrjackson,NaN
1,6.315919e+10,theartcandee,6.307250e+10,gunacar,NaN
2,6.315919e+10,theartcandee,6.975349e+10,tamar.abrams,NaN
3,6.315919e+10,theartcandee,6.346737e+10,elizaa_blogs,NaN
4,6.315919e+10,theartcandee,6.339390e+10,missmatador,NaN
...,...,...,...,...,...
10324,6.340413e+10,ajmstoryteller,7.017196e+10,reeselandrypoetry,NaN
10325,6.340413e+10,ajmstoryteller,6.326141e+10,sethappeal,NaN
10326,6.340413e+10,ajmstoryteller,6.867097e+10,acacia_kamille,NaN
10327,6.340413e+10,ajmstoryteller,6.535157e+10,metal.druid,NaN


In [33]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }
            extracted_posts.append(extracted)
            
            
    return extracted_posts


In [34]:

dfpost = pd.read_csv(dfpostpath)
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

7977

In [35]:
user_processed =list(user_processed)
user_processed.append("68791764906.0")


In [36]:
previus = ""
for idx ,row in df.iterrows():
    if idx < 8660:
        continue
    if str(row["thread_follower_pk"]) == previus:
        print("Processed ", idx, "on ", len(df))
        continue
    if not str(row["thread_follower_pk"]) in user_processed:
        previus = row["thread_follower_pk"]
        print(row["thread_follower_pk"], " ",row["thread_follower_username"])
        try:
            id = get_threads_userpk(row["thread_follower_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_follower_pk"])
                print("Processed ", idx, "on ", len(df))
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))
        if posts is None or  posts["data"] is None or posts["data"]["mediaData"] is None:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        elif not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_follower_pk"]))
        user_processed.append(str(row["thread_follower_pk"]))
        temp_df.to_csv(dfpostpath, mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))
    else:
        print("Processed ", idx, "on ", len(df))

Processed  8660 on  10329
Processed  8661 on  10329
Processed  8662 on  10329
Processed  8663 on  10329
Processed  8664 on  10329
Processed  8665 on  10329
Processed  8666 on  10329
Processed  8667 on  10329
Processed  8668 on  10329
Processed  8669 on  10329
Processed  8670 on  10329
Processed  8671 on  10329
Processed  8672 on  10329
Processed  8673 on  10329
Processed  8674 on  10329
Processed  8675 on  10329
Processed  8676 on  10329
Processed  8677 on  10329
Processed  8678 on  10329
Processed  8679 on  10329
Processed  8680 on  10329
Processed  8681 on  10329
70737503315.0   rayray.creates.empires__
Processed  8682 on  10329
Processed  8683 on  10329
Processed  8684 on  10329
Processed  8685 on  10329
Processed  8686 on  10329
Processed  8687 on  10329
Processed  8688 on  10329
Processed  8689 on  10329
Processed  8690 on  10329
Processed  8691 on  10329
Processed  8692 on  10329
Processed  8693 on  10329
Processed  8694 on  10329
Processed  8695 on  10329
Processed  8696 on  103

In [38]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk,caption_text_translated,sentiment_score,sentiment_label
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,2025-02-02 01:42:22,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Behind the scenes,0.997783,POSITIVE
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\n,0.0,2025-01-31 22:39:13,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,A little over 24 hours left to vote.,0.969585,NEGATIVE
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,2025-01-31 16:26:30,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,"Galaxy Quest \nLots of cutting holes, playing ...",0.868657,NEGATIVE
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,2025-01-31 16:13:49,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.935727,NEGATIVE
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,2025-01-29 05:24:26,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Over due to post videos of my work for @salina...,0.998091,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135003,NaN,NaN,NaN,NaN,NaN,lauraant_,46052227,6.395090e+10,NaN,NaN,NaN,NaN,NaN,NaN
135004,NaN,NaN,NaN,NaN,NaN,romanini.massimo2000,8378076777,6.521294e+10,NaN,NaN,NaN,NaN,NaN,NaN
135005,NaN,NaN,NaN,NaN,NaN,jafar.hoseinzade,5369342807,6.957117e+10,NaN,NaN,NaN,NaN,NaN,NaN
135006,NaN,NaN,NaN,NaN,NaN,lalrinchhana53,48452512234,6.787265e+10,NaN,NaN,NaN,NaN,NaN,NaN
